In [1]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [2]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"
graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-01-24 17:52:30,011:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-01-24 17:52:30,012:connector._init_connection:99] - create driver connection  
[DEBUG:2023-01-24 17:52:31,019:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


In [3]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(max_length=30, trim_whitespaces=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class Roles(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty()
    }

class Programming(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
    
class Game(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(min_length=2, trim_whitespaces=True)
    }
        
class HasSkill(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }

class HasRole(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    
    

In [4]:
from invana.helpers.helpers import install_models

shoud_install_models = True

if shoud_install_models is True:
    # i_understand_rollback=True will roll back any transactions in pending.
    install_models(graph, Person, Programming, Game, HasSkill, Roles, HasRole)
    print("Installed models")
else:
    print(f"Ignoring models installation because install_models={install_models}")

[DEBUG:2023-01-24 17:52:31,057:events.log_event:55] - Request 86bae2ff-faaa-4e13-bb87-ab6e117e0ec1 STARTED with query: mgmt = graph.openManagement()
if (mgmt.containsVertexLabel('Person'))
    Person = mgmt.getVertexLabel('Person')
else 
    Person = mgmt.makeVertexLabel('Person').make()
if (mgmt.containsRelationType('first_name'))
    first_name = mgmt.getPropertyKey('first_name')
else 
    first_name = mgmt.makePropertyKey('first_name').dataType(String.class).make()
if (mgmt.containsRelationType('is_active'))
    is_active = mgmt.getPropertyKey('is_active')
else 
    is_active = mgmt.makePropertyKey('is_active').dataType(Boolean.class).make()
if (mgmt.containsRelationType('created_at'))
    created_at = mgmt.getPropertyKey('created_at')
else 
    created_at = mgmt.makePropertyKey('created_at').dataType(Date.class).make()
mgmt.addProperties(Person, first_name, is_active, created_at)
mgmt.commit();; request_options: {};; at 2023-01-24 17:52:31.057265
[DEBUG:2023-01-24 17:52:36,086:even

Installed models


In [5]:
Person.objects.delete()
Programming.objects.delete()
Game.objects.delete()
HasSkill.objects.delete()
HasRole.objects.delete()

p = Person.objects.create(first_name="rrmerugu")

_, django = Programming.objects.get_or_create(name="Django")
_, python = Programming.objects.get_or_create(name="Python")
_, neo4j = Programming.objects.get_or_create(name="neo4j")
_, janusgraph = Programming.objects.get_or_create(name="Janusgraph")

HasSkill.objects.get_or_create(p.id, django.id)
HasSkill.objects.get_or_create(p.id, python.id)
HasSkill.objects.get_or_create(p.id, neo4j.id)
HasSkill.objects.get_or_create(p.id, janusgraph.id)

_, cod = Game.objects.get_or_create(name="Call of Duty")
_, fifa = Game.objects.get_or_create(name="Fifa")

HasSkill.objects.get_or_create(p.id, cod.id)
HasSkill.objects.get_or_create(p.id, fifa.id)


_, dev_role = Roles.objects.get_or_create(name="Developer")
_, admin_role = Roles.objects.get_or_create(name="Admin")

HasRole.objects.get_or_create(p.id, dev_role.id)
HasRole.objects.get_or_create(p.id, admin_role.id)


print("Created data")

Created data


In [6]:
# p = Person.objects.get_or_none(has__first_name="rrmerugu")
# graph.connector.vertex.search(has__id=p.id).get_traversal() \
# .bothE("has_skill")._as("e") \
# .inV().hasLabel("Programming")._as("v")\
# .select("a","e") \
# .toList()
# # print(p)

In [15]:
from gremlin_python.process.graph_traversal import __

p = Person.objects.get_or_none(has__first_name="rrmerugu")
data = graph.connector.vertex.search(has__id=p.id).get_traversal() \
.bothE("has_skill") \
.inV().hasLabel("Programming") \
.path().by(__.elementMap())
#.toList()

for d in data:
    print("===", d)
# print(p)

=== path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_skill id="169u-2a6o-28lx-18bk" 106512:Person -> has_skill -> 57440:Programming created_at=2023-01-24 17:52:51.479000 >, <Node:Programming id="57440" name=neo4j >]
=== path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_skill id="15vm-2a6o-28lx-2dcg" 106512:Person -> has_skill -> 110608:Programming created_at=2023-01-24 17:52:51.464000 >, <Node:Programming id="110608" name=Python >]
=== path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_skill id="16o2-2a6o-28lx-2gi8" 106512:Person -> has_skill -> 114704:Programming created_at=2023-01-24 17:52:51.493000 >, <Node:Programming id="114704" name=Janusgraph >]
=== path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <Relati

In [29]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeInComingNeighbors(p.id).toList()

[]

In [30]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeOutGoingNeighbors(p.id, "has_role").toList()

[path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_role id="17uq-2a6o-2brp-m74" 106512:Person -> has_role -> 28768:Roles created_at=2023-01-24 17:52:51.675000 >, <Node:Roles id="28768" name=Developer >],
 path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_role id="188y-2a6o-2brp-sl4" 106512:Person -> has_role -> 37048:Roles created_at=2023-01-24 17:52:51.689000 >, <Node:Roles id="37048" name=Admin >]]

In [42]:
p = Person.objects.get_or_none(has__first_name="rrmerugu")
graph.connector.vertex.getNodeOutGoingNeighbors(p.id, "has_skill", neighbors_labels=["Game"]).toList()

[path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_skill id="172a-2a6o-28lx-1bhc" 106512:Person -> has_skill -> 61536:Game created_at=2023-01-24 17:52:51.627000 >, <Node:Game id="61536" name=Call of Duty >],
 path[<Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >, <RelationShip:has_skill id="17gi-2a6o-28lx-oggog" 106512:Person -> has_skill -> 41078896:Game created_at=2023-01-24 17:52:51.645000 >, <Node:Game id="41078896" name=Fifa >]]

In [39]:
# p = Person.objects.get_or_none(has__first_name="rrmerugu")
# graph.connector.vertex.search(has__id=p.id).get_traversal().outE().inV().path().by(__.elementMap()).toList()

In [47]:
p = Programming.objects.get_or_none(has__name="Django")

graph.connector.vertex.getNodeInComingNeighbors(p.id,).toList()        


[path[<Node:Programming id="41074800" name=Django >, <RelationShip:has_skill id="15he-2a6o-28lx-ogdio" 106512:Person -> has_skill -> 41074800:Programming created_at=2023-01-24 17:52:51.446000 >, <Node:Person id="106512" first_name=rrmerugu created_at=2023-01-24 17:52:51.296000 is_active=True >]]